# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [37]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact

In [38]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("MAP_BOX_API")
print(map_box_api)
px.set_mapbox_access_token(map_box_api)

pk.eyJ1Ijoicm9taWxwYXJla2giLCJhIjoiY2tzcDltZGduMDB4YTJucWtuYTZ6c3hpbSJ9.EhBBPzvy2tmTYXc3ZioPig


# Import Data

In [39]:
# Import the necessary CSVs to Pandas DataFrames
# YOUR CODE HERE!
file_path = Path("Data/neighborhoods_coordinates.csv")
coordinatesData = pd.read_csv(file_path).dropna()
coordinatesData.head(5)

,Neighborhood,Lat,Lon
0,Alamo Square,37.791012,-122.402100
1,Anza Vista,37.779598,-122.443451
2,Bayview,37.734670,-122.401060
3,Bayview Heights,37.728740,-122.410980
4,Bernal Heights,37.728630,-122.443050


In [40]:
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path,index_col="year")
#sfo_data.reset_index(inplace = True)
sfo_data.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2010,Alamo Square,291.182946,372560,1239
2010,Anza Vista,267.932583,372560,1239
2010,Bayview,170.098665,372560,1239
2010,Buena Vista Park,347.394919,372560,1239
2010,Central Richmond,319.027623,372560,1239


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [59]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    # YOUR CODE HERE!
    meanOfHousingUnitsPerYear = (sfo_data["housing_units"].groupby(['year']).mean())
    barPlotOfMeanOfHousingUnitsPerYear = meanOfHousingUnitsPerYear.hvplot.bar(x='year', y= 'housing_units',ylim = (370000,390000)).opts(yformatter="%.0f", title = 'Housig Units in San Fransisco 2010-2016')
    return barPlotOfMeanOfHousingUnitsPerYear

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    # YOUR CODE HERE!
    avgPricePerSqftAndGrossRent = (sfo_data.groupby(['year']).mean())
    linePlotGrossRent = avgPricePerSqftAndGrossRent["gross_rent"].hvplot(label = 'Gross Rent' )
    return linePlotGrossRent


def average_sales_price():
    """Average Sales Price Per Year.""" 
    # YOUR CODE HERE!
    avgPricePerSqftAndGrossRent = (sfo_data.groupby(['year']).mean())
    linePlotSalePricePerSqft = avgPricePerSqftAndGrossRent['sale_price_sqr_foot'].hvplot(label = 'Sale Price Per Square Foot' )
    return linePlotSalePricePerSqft


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!
    avgPricePerSqftByNeighborhood = (sfo_data.groupby(['year','neighborhood']).mean())
    interactivePlotAvgPricePerSqftByNeighborhood =  avgPricePerSqftByNeighborhood['gross_rent'].hvplot.line(x= 'year', y = 'gross_rent',title = 'Average Monthly Rent',groupby = 'neighborhood')
    return interactivePlotAvgPricePerSqftByNeighborhood
    
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    # YOUR CODE HERE!
    averageNeighborhood = (sfo_data.groupby(['neighborhood']).mean())
    top10Neighborhood = averageNeighborhood.nlargest(10,'sale_price_sqr_foot') 
    top10Neighborhood.reset_index()
    barPlotTop10Neighborhood =  top10Neighborhood.hvplot.bar(x= 'neighborhood', y = 'sale_price_sqr_foot',title = 'Top 10 Neighbourhood' )
    return barPlotTop10Neighborhood

def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    # YOUR CODE HERE!
    avgPricePerSqftByNeighborhood = (sfo_data.groupby(['year','neighborhood']).mean())
    avgPricePerSqftByNeighborhood.drop('housing_units', axis = 1, inplace = True )
    compareBarPlot = avgPricePerSqftByNeighborhood.hvplot.bar(x= 'year',title = 'Sale Price Per Square Foot',groupby = 'neighborhood',rot  = 90 )
    return compareBarPlot
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    # YOUR CODE HERE!
    averageNeighborhood = (sfo_data.groupby(['neighborhood']).mean())
    top10Neighborhood = averageNeighborhood.nlargest(10,'sale_price_sqr_foot') 
    top10Neighborhood.reset_index(inplace = True)
    parallelCoordinatesPlot = px.parallel_coordinates(top10Neighborhood, color='sale_price_sqr_foot',width = 1000, title = 'Parallel Coordinates Analysis of most expensive neighborhood in San Fransisco')
    return parallelCoordinatesPlot


def parallel_categories():
    """Parallel Categories Plot."""
    averageNeighborhood = (sfo_data.groupby(['neighborhood']).mean())
    top10Neighborhood = averageNeighborhood.nlargest(10,'sale_price_sqr_foot') 
    top10Neighborhood.reset_index(inplace = True)
    parallelCategoriesPlot = px.parallel_categories(
    top10Neighborhood,
    dimensions=['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'],
    color='sale_price_sqr_foot',width = 1500, title = 'Parallel Categoris Analysis of most expensive neighborhood in San Fransisco')
    return parallelCategoriesPlot


def neighborhood_map():
    """Neighborhood Map."""

    # YOUR CODE HERE!
    meanPricePerSqftByNeighborhood = (sfo_data.groupby(['neighborhood']).mean())
    meanPricePerSqftByNeighborhood.reset_index(inplace = True)
    avgNeighborhoodLocation = pd.concat([coordinatesData,meanPricePerSqftByNeighborhood],axis = 1 , join = 'inner' )
    avgNeighborhoodLocation.drop('neighborhood',axis =1 , inplace = True)
    px.set_mapbox_access_token(map_box_api)
    map_1 = px.scatter_mapbox(
    avgNeighborhoodLocation,
    lat="Lat",
    lon="Lon",
    color="gross_rent",
    title = 'Neighborhood Map'
    )
    return map_1

def sunburst():
    """Sunburst Plot."""
    
    # YOUR CODE HERE!
    avgPricePerSqftByNeighborhood = (sfo_data.groupby(['year','neighborhood']).mean())
    avgPricePerSqftByNeighborhood.reset_index(inplace = True)
    fig = px.sunburst(avgPricePerSqftByNeighborhood ,path  =  ['year', 'neighborhood'],values = 'gross_rent',color = 'gross_rent', title = 'Cost Analysis of most expensive neighborhood in San Fransisco by Year')
    return fig

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [67]:
# Create a Title for the Dashboard
# YOUR CODE HERE!
#row_of_bar = pn.Row(parallel_coordinates())
#row_of_bar.append(parallel_categories())
rowOfParallel = pn.Row(housing_units_per_year(), average_gross_rent(),average_sales_price())
colOfParallel = pn.Column(parallel_coordinates(), parallel_categories())
colOfParallelNeighborhood = pn.Column(average_price_by_neighborhood(),most_expensive_neighborhoods_rent_sales(),top_most_expensive_neighborhoods())

# Create a tab layout for the dashboard
# YOUR CODE HERE!
tabs = pn.Tabs(
    ("Welcome", neighborhood_map()),
    ("Yearly Market Analysis", rowOfParallel),
    ("Neighborhood Analysis", colOfParallelNeighborhood),
    ("Parallel Plot Analysis", colOfParallel),
    ("Suburst Plot Analysis", sunburst())
    
)
tabs

# Create the dashboard
# YOUR CODE HERE!

Tabs
    [0] Plotly(Figure, name='Welcome')
    [1] Row
        [0] HoloViews(Bars)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
    [2] Column
        [0] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [2] HoloViews(Bars)
    [3] Column
        [0] Plotly(Figure, visible=False)
        [1] Plotly(Figure, visible=False)
    [4] Plotly(Figure, name='Suburst Plot Analysis', visible=False)

## Serve the Panel Dashboard

In [43]:
# Serve the# dashboard
# YOUR CODE HERE!

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [44]:
#housing_units_per_year()

In [45]:
#average_gross_rent()

In [46]:
#average_sales_price()

In [47]:
#average_price_by_neighborhood()

In [48]:
#top_most_expensive_neighborhoods()

In [49]:
#most_expensive_neighborhoods_rent_sales()

In [50]:
#neighborhood_map()

In [51]:
#parallel_categories()

In [52]:
#parallel_coordinates()

In [53]:
#sunburst()